In [ ]:
import numpy as np
import pandas as pd

def iou_with_anchors(anchors_min, anchors_max, box_min, box_max):
    """Compute jaccard score between a box and the anchors."""

    len_anchors = anchors_max - anchors_min
    int_xmin = np.maximum(anchors_min, box_min)
    int_xmax = np.minimum(anchors_max, box_max)
    inter_len = np.maximum(int_xmax - int_xmin, 0.0)
    union_len = len_anchors - inter_len + box_max - box_min
    iou = inter_len / union_len
    return iou


def soft_nms(df, alpha, t1, t2, fps):
    df = df.sort_values(by="score", ascending=False)
    t_start = list(df.begin.values[:] / fps)
    t_end = list(df.end.values[:] / fps)
    t_score = list(df.score.values[:])

    r_start = []
    r_end = []
    r_score = []

    while len(t_score) > 1 and len(r_score) < 101:
        max_index = t_score.index(max(t_score))
        tmp_iou_list = iou_with_anchors(
            np.array(t_start), np.array(t_end), t_start[max_index], t_end[max_index]
        )
        for idx in range(0, len(t_score)):
            if idx != max_index:
                tmp_iou = tmp_iou_list[idx]
                tmp_width = t_end[max_index] - t_start[max_index]
                if tmp_iou > t1 + (t2 - t1) * tmp_width:
                    t_score[idx] *= np.exp(-np.square(tmp_iou) / alpha)

        r_start.append(t_start[max_index])
        r_end.append(t_end[max_index])
        r_score.append(t_score[max_index])
        t_start.pop(max_index)
        t_end.pop(max_index)
        t_score.pop(max_index)

    new_df = pd.DataFrame()
    new_df["score"] = r_score
    new_df["begin"] = r_start
    new_df["end"] = r_end

    new_df["begin"] *= fps
    new_df["end"] *= fps
    return new_df

In [3]:
df = pd.read_csv('test.csv')

In [6]:
soft_nms(df,  fps=40, alpha=0.4, t1=0.2, t2=0.9)

score  begin    end
0    0.481676   56.0   74.0
1    0.481646   92.0  122.0
2    0.481603   65.0   86.0
3    0.481563   10.0   12.0
4    0.481533   26.0   50.0
..        ...    ...    ...
96   0.481280   13.0   72.0
97   0.481280    6.0   58.0
98   0.481280  104.0  154.0
99   0.481279   49.0  108.0
100  0.481278    6.0   64.0

[101 rows x 3 columns]